<a href="https://colab.research.google.com/github/pranav-vijayananth/SOMResearch/blob/main/tensorflow_som_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/cgorman/tensorflow-som/master/tf_som.py

In [ ]:
!pip3 install tensorflow==1.15

In [ ]:
import numpy as np

np.set_printoptions(suppress=True)

import math
import tensorflow as tf
from tf_som import SelfOrganizingMap
from scipy.spatial import distance_matrix
import subprocess
from collections import Counter
import time 
import logging

print(tf.__version__)

In [ ]:
#FUNCTION FOR GETTING HEAT MAP OF CLUSTERS AND THE BMU LOCATIONS
def get_umatrix(input_vects, weights, m, n):
    umatrix = np.zeros((m * n, 1))
    neuron_locs = list()
    for i in range(m):
        for j in range(n):
            neuron_locs.append(np.array([i, j]))

    neuron_distmat = distance_matrix(neuron_locs, neuron_locs)

    for i in range(m * n):
        neighbor_idxs = neuron_distmat[i] <= 1  
        neighbor_weights = weights[neighbor_idxs]
        umatrix[i] = distance_matrix(np.expand_dims(weights[i], 0), neighbor_weights).mean()

    bmu_indices = []
    for vect in input_vects:
        min_index = min([i for i in range(len(list(weights)))],
                        key=lambda x: np.linalg.norm(vect-
                                                     list(weights)[x]))
        bmu_indices.append(neuron_locs[min_index])
        
    return umatrix, bmu_indices

In [ ]:
data = []
groundtruth = []
dataread = datafile.readline()
dataread = datafile.readline()

while dataread != "": 
  a = dataread.split(",")
  l2 = []
  for j in range(0, len(a), 1):
    if j == len(a)-1:
      groundtruth.append(a[j].strip())
    else:
      try: 
        l2.append(float(a[j]))
      except:
        l2.append(0)
  data.append(l2)
  dataread = datafile.readline()

rows = len(data)
cols = len(data[0])
num_inputs = rows * cols

datafile.close()

In [ ]:
print(f"This is the data: {data}")
print(f"Number of cols: {cols}")
print(f"These are the number of rows {rows}")
print(f"This is the grountruth: {groundtruth}")
print(f"Number of input vectors: {num_inputs}")

#CLUSTERS
uniqueValues = Counter(groundtruth).keys()
clusters = len(uniqueValues)
print(f"The clusters in dataset: {clusters}")

In [ ]:
#CREATING THE TENSORFLOW GRAPH AND LOGS FOR THE MODEL

# for i in range(1, 30+1):
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
graph = tf.Graph()

with graph.as_default():
  session = tf.Session(config=tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False
  ))

  #BATCH SIZE 
  batch_size = 32

  #CONVERT TO NUMPY ARRAY FOR TF DATA PIPELINE
  data = np.array(data, dtype="float32")

  #MAKING THE TENSORFLOW DATRASET PIPELINE 
  input_data = tf.data.Dataset.from_tensor_slices(data)
  input_data = input_data.repeat()
  input_data = input_data.batch(batch_size)
  iterator = input_data.make_one_shot_iterator()
  next_element = iterator.get_next()

  #BUILDING THE SOM OBJECT
  som = SelfOrganizingMap(m=clusters, n=clusters, dim=cols, max_epochs=20, session=session, graph=graph, input_tensor=next_element, batch_size=batch_size, initial_learning_rate=0.1, model_name='Self-Organizing-Map', softmax_activity=True)

  #MAKING + RUNNING SESSION
  init_op = tf.global_variables_initializer()
  session.run([init_op])

  #TRAINING
  som.train(num_inputs)

  #WEIGHTS
  som_weights = som.output_weights

  #SOM UMATRIX + BMU LOCATION/PREDICTION
  umatrix, bmu_loc = get_umatrix(data, som_weights, clusters, clusters)